<a href="https://colab.research.google.com/github/Divyavavilthota/chat_with_pdf/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install PyPDF2 sentence-transformers faiss-cpu openai
!pip install PyMuPDF
!pip install PyMuPDF
!pip install tools
!pip install fastapi
!pip install frontend


In [64]:
import os
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
from whoosh.index import open_dir
from PyPDF2 import PdfReader

# Ensure the directory for the index exists
index_dir = '/content/pdf_index/'
if not os.path.exists(index_dir):
    os.makedirs(index_dir)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text()
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text

# Function to get the number of pages in the PDF
def get_pdf_page_count(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        return len(reader.pages)
    except Exception as e:
        print(f"Error getting page count from PDF: {e}")
        return 0  # Return 0 if there's an error

# Function to search the text in the index
def search_text(query_string):
    ix = open_dir(index_dir)  # Use the directory with the index
    with ix.searcher() as searcher:
        query = QueryParser("content", ix.schema).parse(query_string)
        results = searcher.search(query, limit=5)  # Limit results to top 5
        response = []
        for result in results:
            response.append({
                'content': result['content'],
                'score': result.score,
                'highlights': result.highlights("content")  # Show highlighted match
            })
        return response

# Example usage
pdf_path = "/content/sample.pdf"
extracted_text = extract_text_from_pdf(pdf_path)

# Check if text is extracted
print("Extracted Text: ", extracted_text[:500])  # Print first 500 characters to verify

# Get the number of pages in the PDF
page_count = get_pdf_page_count(pdf_path)
print(f"Number of pages in the PDF: {page_count}")

# Create an index
schema = Schema(content=TEXT(stored=True))
ix = create_in(index_dir, schema)
writer = ix.writer()

# Check if the document is being indexed properly
if extracted_text.strip():  # Only index if there's text to index
    print("Indexing document: ", extracted_text[:500])  # Print first 500 characters
    writer.add_document(content=extracted_text)
    writer.commit()
else:
    print("No text extracted from PDF, skipping indexing.")

# Query the index with a more specific or broader query (e.g., 'What is the primary objective of the study?')
query = "what is abstract?"  # Update with a relevant question
response = search_text(query)

# Output the response
if response:
    print("Response:")
    for idx, res in enumerate(response, 1):
        print(f"Result {idx}:")
        print(f"Score: {res['score']}")
        print(f"Content: {res['content'][10:]}...")  # Display first 500 characters of result
        print(f"Highlights: {res['highlights']}")
else:
    print("No results found for the query.")


Extracted Text:  Estimating
Countries
with
Similar
Maternal
Mortality
Rate
using
Cluster
Analysis
and
Pairing
Countries
with
Identical
MMR
Dr.
S.
Nandini
Associate
Professor
in
Zoology
Quaid-E-Millath
Government
College
for
Women,
Anna
Salai,
Chennai
-
600
002
s.nandini0507@gmail.com
Sanjjushri
Varshini
R
sanjjushrivarshini@gmail.com
Abstract:
In
the
evolving
world,
we
require
more
additionally
the
young
era
to
flourish
and
evolve
into
developed
land.
Most
of
the
population
all
around
the
world
are
unaware
of
th
Number of pages in the PDF: 14
Indexing document:  Estimating
Countries
with
Similar
Maternal
Mortality
Rate
using
Cluster
Analysis
and
Pairing
Countries
with
Identical
MMR
Dr.
S.
Nandini
Associate
Professor
in
Zoology
Quaid-E-Millath
Government
College
for
Women,
Anna
Salai,
Chennai
-
600
002
s.nandini0507@gmail.com
Sanjjushri
Varshini
R
sanjjushrivarshini@gmail.com
Abstract:
In
the
evolving
world,
we
require
more
additionally
the
young
era
to
flourish
and
evolve
into
develope

In [68]:
import os
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
from whoosh.index import open_dir
from PyPDF2 import PdfReader
import re

# Ensure the directory for the index exists
index_dir = '/content/pdf_index/'
if not os.path.exists(index_dir):
    os.makedirs(index_dir)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text()
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text

# Function to get the number of pages in the PDF
def get_pdf_page_count(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        return len(reader.pages)
    except Exception as e:
        print(f"Error getting page count from PDF: {e}")
        return 0  # Return 0 if there's an error

# Function to extract the abstract from the document
def extract_abstract(text):
    # Look for text that typically starts an abstract section with more flexibility
    match = re.search(r"(?i)(abstract|summary)[^:]*[:\s]*\n*(.*?)\n+(?:Introduction|1\.)", text, re.DOTALL)
    if match:
        return match.group(2).strip()
    else:
        return "Abstract not found in the document."

# Function to search the text in the index
def search_text(query_string):
    ix = open_dir(index_dir)  # Use the directory with the index
    with ix.searcher() as searcher:
        query = QueryParser("content", ix.schema).parse(query_string)
        results = searcher.search(query, limit=5)  # Limit results to top 5
        response = []
        for result in results:
            response.append({
                'content': result['content'],
                'score': result.score,
                'highlights': result.highlights("content")  # Show highlighted match
            })
        return response

# Function to answer specific questions
def answer_question(query_string, pdf_path, extracted_text):
    # Check if the query is about the number of pages in the PDF
    if "how many pages" in query_string.lower():
        page_count = get_pdf_page_count(pdf_path)
        return f"The number of pages in the document is: {page_count}"

    # Check if the query is about the abstract
    if "abstract" in query_string.lower():
        abstract = extract_abstract(extracted_text)
        return f"Abstract: {abstract}"

    # If the question is not about the page count or abstract, search the text in the index
    response = search_text(query_string)
    if response:
        return f"Found the following related information:\n" + "\n".join([res['content'][:500] for res in response])  # Show first 500 characters
    else:
        return "No relevant information found for your query."

# Example usage
pdf_path = "sample.pdf"
extracted_text = extract_text_from_pdf(pdf_path)

# Check if text is extracted
if extracted_text.strip():  # Only index if there's text to index
    # Create an index and add document
    schema = Schema(content=TEXT(stored=True))
    ix = create_in(index_dir, schema)
    writer = ix.writer()
    writer.add_document(content=extracted_text)
    writer.commit()
else:
    print("No text extracted from PDF, skipping indexing.")

# Ask a question related to the document
query = "What is the abstract and keywords?"
response = answer_question(query, pdf_path, extracted_text)

# Output the response
print(response)


Abstract: In
the
evolving
world,
we
require
more
additionally
the
young
era
to
flourish
and
evolve
into
developed
land.
Most
of
the
population
all
around
the
world
are
unaware
of
the
complications
involved
in
the
routine
they
follow
while
they
are
pregnant
and
how
hospital
facilities
affect
maternal
health.
Maternal
Mortality
is
the
death
of
a
pregnant
woman
due
to
intricacies
correlated
to
pregnancy,
underlying
circumstances
exacerbated
by
the
pregnancy
or
management
of
these
situations.
It
is
crucial
to
consider
the
Maternal
Mortality
Rate
(MMR)
in
diverse
locations
and
determine
which
human
routines
and
hospital
facilities
diminish
the
Maternal
Mortality
Rate
(MMR).
This
research
aims
to
examine
and
discover
the
countries
which
are
keeping
more
lavish
threats
of
MMR
and
countries
alike
in
MMR
encountered.
Data
is
examined
and
collected
for
various
countries,
data
consists
of
the
earlier
years'
observation.
From
the
perspective
of
Machine
Learning,
Unsupervised
Machine
Learning
is
im

# New Section

In [54]:
extracted_text = extract_text_from_pdf(pdf_path)
print("Extracted Text: ", extracted_text[:500])  # Print the first 500 characters to verify


Extracted Text:  Estimating
Countries
with
Similar
Maternal
Mortality
Rate
using
Cluster
Analysis
and
Pairing
Countries
with
Identical
MMR
Dr.
S.
Nandini
Associate
Professor
in
Zoology
Quaid-E-Millath
Government
College
for
Women,
Anna
Salai,
Chennai
-
600
002
s.nandini0507@gmail.com
Sanjjushri
Varshini
R
sanjjushrivarshini@gmail.com
Abstract:
In
the
evolving
world,
we
require
more
additionally
the
young
era
to
flourish
and
evolve
into
developed
land.
Most
of
the
population
all
around
the
world
are
unaware
of
th


In [57]:
import os
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
from whoosh.index import open_dir
from PyPDF2 import PdfReader

# Ensure the directory for the index exists
index_dir = '/content/pdf_index/'
if not os.path.exists(index_dir):
    os.makedirs(index_dir)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text()
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text

# Function to search the text in the index
def search_text(query_string):
    ix = open_dir(index_dir)  # Use the directory with the index
    with ix.searcher() as searcher:
        query = QueryParser("content", ix.schema).parse(query_string)
        results = searcher.search(query)
        return [result['content'] for result in results]

# Example usage
pdf_path = "/content/sample.pdf"
extracted_text = extract_text_from_pdf(pdf_path)

# Check if text is extracted
print("Extracted Text: ", extracted_text[:500])  # Print first 500 characters to verify

# Create an index
schema = Schema(content=TEXT(stored=True))
ix = create_in(index_dir, schema)
writer = ix.writer()

# Check if the document is being indexed properly
if extracted_text.strip():  # Only index if there's text to index
    print("Indexing document: ", extracted_text[:500])  # Print first 500 characters
    writer.add_document(content=extracted_text)
    writer.commit()
else:
    print("No text extracted from PDF, skipping indexing.")

# Query the index with a broader query
query = "how many pages are there?"
response = search_text(query)

# Output the response
print("Response:", response)


Extracted Text:  Estimating
Countries
with
Similar
Maternal
Mortality
Rate
using
Cluster
Analysis
and
Pairing
Countries
with
Identical
MMR
Dr.
S.
Nandini
Associate
Professor
in
Zoology
Quaid-E-Millath
Government
College
for
Women,
Anna
Salai,
Chennai
-
600
002
s.nandini0507@gmail.com
Sanjjushri
Varshini
R
sanjjushrivarshini@gmail.com
Abstract:
In
the
evolving
world,
we
require
more
additionally
the
young
era
to
flourish
and
evolve
into
developed
land.
Most
of
the
population
all
around
the
world
are
unaware
of
th
Indexing document:  Estimating
Countries
with
Similar
Maternal
Mortality
Rate
using
Cluster
Analysis
and
Pairing
Countries
with
Identical
MMR
Dr.
S.
Nandini
Associate
Professor
in
Zoology
Quaid-E-Millath
Government
College
for
Women,
Anna
Salai,
Chennai
-
600
002
s.nandini0507@gmail.com
Sanjjushri
Varshini
R
sanjjushrivarshini@gmail.com
Abstract:
In
the
evolving
world,
we
require
more
additionally
the
young
era
to
flourish
and
evolve
into
developed
land.
Most
of
the
population
